In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss,roc_auc_score,roc_curve,accuracy_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LinearRegression ,LogisticRegression
from sklearn.ensemble import RandomForestClassifier,StackingClassifier
from sklearn.model_selection import StratifiedKFold,cross_val_score,KFold
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score 
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
import  os

In [63]:

kyp = pd.read_csv("C:/Users\dai\Desktop\Machine Learning/Cases/Cases/Kyphosis/Kyphosis.csv")
le = LabelEncoder()

y = le.fit_transform(kyp['Kyphosis'])
X = kyp.drop('Kyphosis',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                   test_size = 0.3, 
                                   random_state=24,
                                   stratify = y)



In [64]:
lr=LogisticRegression()
svm=SVC(kernel='linear', probability=True,random_state=24)
dtc= DecisionTreeClassifier(random_state=24)
rf= RandomForestClassifier(random_state=24)
stack=StackingClassifier([('LR',lr),('SVM',svm),('TREE',dtc)],final_estimator=rf)

stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = stack.predict_proba(X_test)
print(log_loss(y_test, y_pred_proba))

0.72
2.001191391953967


In [65]:
####with pass through
stack=StackingClassifier([('LR',lr),('SVM',svm),('TREE',dtc)],final_estimator=rf,passthrough=True)

stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = stack.predict_proba(X_test)
print(log_loss(y_test, y_pred_proba))

0.76
1.8078240247569928


In [66]:
#print(stack.get_params() )
params= {'SVM__C':np.linspace(0.001, 3,5),
        'LR__C':np.linspace(0.001, 3,5),
        'TREE__max_depth':[None,2,3,4],
        'final_estimator__max_features':[2,3],
        'passthrough':[False,True]
        }
kfold = StratifiedKFold(n_splits = 5,shuffle=True,random_state=24)

In [ ]:
gcv = GridSearchCV(stack, param_grid=params, cv=kfold ,scoring='neg_log_loss')
gcv.fit(X, y)


In [ ]:
print(gcv.best_params_)
print(gcv.best_score_)